# inferencer

> Infers the schedule
- order: 9

In [ ]:
#| default_exp inferencer

In [ ]:
!which python

/Users/kobus/aishiftscheduler/.venv/bin/python


In [ ]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import defaultdict
# import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
import time
# import math
# from pprint import pprint
## !pip install -U "ray"
# import ray
# import json

import aishiftscheduler.config as cf
import aishiftscheduler.model as mod
import aishiftscheduler.policy as pol
import aishiftscheduler.utils as utl

In [ ]:
#| export
def get_best_theta_Alloc():
  best_theta_Alloc = \
    pol.Policy(None).build_theta({
      'thCumSlots': cf.BEST_THETA_Alloc[0],
      'thSickProb': cf.BEST_THETA_Alloc[1],
      'thCumMerits': cf.BEST_THETA_Alloc[2],
      'thContSlots': cf.BEST_THETA_Alloc[3],
      'thSelect': cf.BEST_THETA_Alloc[4]
    })
  return best_theta_Alloc

In [ ]:
#| export
def do_infer(L, T, best_theta_Alloc, pars):
  M = mod.Model(pars)
  P = pol.Policy(M)
  ## DEM = DemandSimulator(seed=SEED_EVALU)
  ## MER = MeritSimulator(seed=SEED_EVALU)
    
  # thetasOpt = []; thetasOpt.append(get_best_theta_Alloc()) ##from storage
  thetasOpt = []; thetasOpt.append(best_theta_Alloc) ##from storage
  return \
    P.perform_grid_search_sample_paths('X__Alloc', thetasOpt, L, T, pars)

In [ ]:
#| export
from io import StringIO
def prepare_schedule_shifts(df, buf, pars):
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  buf.write(f"SCHEDULE SHIFTS:\n")
  buf.write(f"================\n")
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    buf.write(f'\n************** {resName}:\n')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ts_1 = sched_list[0]
      dow_1 = sched_list[0].day_of_week
      buf.write(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1:
          buf.write(f"{(ts_1).strftime('%a %b %d %Hh%M')}\n\n")
          buf.write(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
        dow_1 = dow
        ts_1 = ts
      buf.write(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}\n")
  buf.write(f'\n{cf.CONTIGUOUS_REWARD=}\n')
  buf.write(f'{pars.MAX_DAILY_SLOT_RUN=} ({pars.RESOLUTION}s)\n')
  buf.write(f'{cf.TH_CumSlots_SPEC=}\n')
  buf.write(f'{cf.TH_SickProb_SPEC=}\n')
  buf.write(f'{cf.TH_CumMerits_SPEC=}\n')
  buf.write(f'{cf.TH_ContSlots_SPEC=}\n')
  buf.write(f'{cf.TH_Select_SPEC=}\n')
  return buf.getvalue()

In [ ]:
#| export
from io import StringIO
def prepare_schedule_slots(df, buf):
  gap_mins = utl.gap_minutes(cf.RESOLUTION)
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  buf.write(f"SCHEDULE SLOTS:\n")
  buf.write(f"===============\n")
  n_gaps = 0
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    buf.write(f'\n************** {resName}:\n')
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ## print each slot and indicate gaps
      ts_1 = sched_list[0]
      dow_1 = -1
      for ts in sched_list:
        dow = ts.day_of_week
        if dow != dow_1: buf.write('\n')
        if (int((ts.to_datetime64() - ts_1.to_datetime64())/(1e9*60)) > gap_mins)\
          and (dow == dow_1):
          n_gaps += 1
          buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP\n")
        else:
          buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
        dow_1 = dow
        ts_1 = ts
  buf.write(f'\nTOTAL NUMBER OF GAPS FOR ALL RESOURCES: {n_gaps}\n')
  buf.write(f'{cf.CONTIGUOUS_REWARD=}\n')
  buf.write(f'{cf.MAX_DAILY_SLOT_RUN=} ({cf.RESOLUTION}s)\n')
  buf.write(f'{cf.TH_CumSlots_SPEC=}\n')
  buf.write(f'{cf.TH_SickProb_SPEC=}\n')
  buf.write(f'{cf.TH_CumMerits_SPEC=}\n')
  buf.write(f'{cf.TH_ContSlots_SPEC=}\n')
  buf.write(f'{cf.TH_Select_SPEC=}\n')
  return buf.getvalue()

In [ ]:
#| export
def prepare_schedule_slots_as_json(df, pars):
  sched_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
  gap_mins = utl.gap_minutes(pars.RESOLUTION)
  mask = df.columns.str.contains('Allocd_t')
  resource_allocs = list(df.columns[mask])
  sched = copy(df)
  schedule = sched[['dt']+resource_allocs]

  n_gaps = 0
  for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    sched_list = list(schedule.loc[
      schedule[res_alloc] == True,
      ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
      ## capture each slot and indicate gaps
      ts_1 = sched_list[0]
      dow_1 = -1
      for ts in sched_list:
        dow = ts.day_of_week
        if (int((ts.to_datetime64() - ts_1.to_datetime64())/(1e9*60)) > gap_mins)\
          and (dow == dow_1):
          n_gaps += 1
          sched_dict['slots'][resName][dow].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP")
        else:
          sched_dict['slots'][resName][dow].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        dow_1 = dow
        ts_1 = ts
  sched_dict['config']['TOTAL NUMBER OF GAPS FOR ALL RESOURCES'] = n_gaps
  sched_dict['config']['CONTIGUOUS_REWARD'] = cf.CONTIGUOUS_REWARD
  sched_dict['config']['MAX_DAILY_SLOT_RUN'] = pars.MAX_DAILY_SLOT_RUN
  sched_dict['config']['TH_CumSlots_SPEC'] = cf.TH_CumSlots_SPEC
  sched_dict['config']['TH_SickProb_SPEC'] = cf.TH_SickProb_SPEC
  sched_dict['config']['TH_CumMerits_SPEC'] = cf.TH_CumMerits_SPEC
  sched_dict['config']['TH_ContSlots_SPEC'] = cf.TH_ContSlots_SPEC
  sched_dict['config']['TH_Select_SPEC'] = cf.TH_Select_SPEC
  return sched_dict

In [ ]:
#| export
def infer_schedule(L, T, First_n_t, stored_best_theta, pars):
    start = time.time()
    
    ThetaStar_expCbarcum_evalu_opt, ThetaStar_expCtilcum_evalu_opt, \
    _, _, \
    Best_theta_evalu_opt, Worst_theta_evalu_opt, \
    _, _, \
    _, _, \
    Record_evalu_opt = \
      do_infer(L, T, stored_best_theta, pars)
    Df_evalu_opt = pd.DataFrame.from_records(
        Record_evalu_opt[:First_n_t], columns=pars.LABELS)
    ## print(
    ##   f'{ThetaStar_expCbarcum_evalu_opt.iloc[-1]=:.2f}')    
    ## print_schedule_shifts(Df_evalu_opt)
    end = time.time(); print(f'EXECUTION TIME: {end - start} seconds')

    buf = StringIO()
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # prepped_sched = prepare_schedule_shifts(Df_evalu_opt, buf, pars)
    prepped_sched = prepare_schedule_slots_as_json(Df_evalu_opt, pars)
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    buf.close()

    return Best_theta_evalu_opt, Df_evalu_opt, prepped_sched

In [ ]:
# ////////////////////////////////////////////

In [ ]:
import aishiftscheduler.parameters as par
pars = par.Parameters()


len(self.RESOURCE_TYPES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(self.TYPES)=13
['Manager', 'AssistMngr', 'AssistMngr', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc', 'RetailAssoc']
len(self.aNAMES)=13
['Manager_Matt', 'AssistMngr_Mike', 'AssistMngr_Tanner', 'RetailAssoc_Jake', 'RetailAssoc_James', 'RetailAssoc_Jane', 'RetailAssoc_John', 'RetailAssoc_Jim', 'RetailAssoc_Jenny', 'RetailAssoc_Jeremy', 'RetailAssoc_Judy', 'RetailAssoc_Julie', 'RetailAssoc_Jeffrey']

len(self.bNAMES)=3
['Manager', 'AssistMngr', 'RetailAssoc']

len(self.abNAMES)=13
['Manager_Matt___Manager', 'AssistMngr_Mike___AssistMngr', 'AssistMngr_Tanner___AssistMngr', 'RetailAssoc_Jake___RetailAssoc', 'RetailAssoc_James___RetailAssoc', 'RetailAssoc_Jane___RetailAssoc', 'RetailAssoc_John___RetailAssoc', 'RetailAssoc_Jim___RetailAssoc', 'RetailAssoc_Jenny___RetailAssoc', 'RetailAssoc_Jeremy___RetailAssoc', 'RetailAssoc_Judy___

In [ ]:
L = 1
T = 7*pars.SLOTS_PER_DAY
First_n_t = int(1*L*T)
stored_best_theta = get_best_theta_Alloc()

ThetaStar_expCbarcum_evalu_opt, ThetaStar_expCtilcum_evalu_opt, \
_, _, \
Best_theta_evalu_opt, Worst_theta_evalu_opt, \
_, _, \
_, _, \
Record_evalu_opt = \
    do_infer(L, T, stored_best_theta, pars)
Df_evalu_opt = pd.DataFrame.from_records(
    Record_evalu_opt[:First_n_t], columns=pars.LABELS)

numThetas=1
... printing every 1th theta (if considered) ...
	%%% l=1
0/0, 443,732, Theta(thCumSlots=0, thSickProb=0, thCumMerits=0, thContSlots=1, thSelect='random')


/Users/kobus/aishiftscheduler/aishiftscheduler/policy.py:212: RuntimeWarning: invalid value encountered in scalar divide
  Ctilcum_tmp = np.sum(np.square(np.array(CcumIomega__lI) - Cbarcum_tmp))/(L - 1)


In [ ]:
Df_evalu_opt.head()

,piName,theta,l,t,dt,RAvail_t_Manager_Matt,RAvail_t_AssistMngr_Mike,RAvail_t_AssistMngr_Tanner,RAvail_t_RetailAssoc_Jake,RAvail_t_RetailAssoc_James,RAvail_t_RetailAssoc_Jane,RAvail_t_RetailAssoc_John,RAvail_t_RetailAssoc_Jim,RAvail_t_RetailAssoc_Jenny,RAvail_t_RetailAssoc_Jeremy,RAvail_t_RetailAssoc_Judy,RAvail_t_RetailAssoc_Julie,RAvail_t_RetailAssoc_Jeffrey,RCumSlots_t_Manager_Matt,RCumSlots_t_AssistMngr_Mike,RCumSlots_t_AssistMngr_Tanner,RCumSlots_t_RetailAssoc_Jake,RCumSlots_t_RetailAssoc_James,RCumSlots_t_RetailAssoc_Jane,RCumSlots_t_RetailAssoc_John,RCumSlots_t_RetailAssoc_Jim,RCumSlots_t_RetailAssoc_Jenny,RCumSlots_t_RetailAssoc_Jeremy,RCumSlots_t_RetailAssoc_Judy,RCumSlots_t_RetailAssoc_Julie,RCumSlots_t_RetailAssoc_Jeffrey,RCumMerits_t_Manager_Matt,RCumMerits_t_AssistMngr_Mike,RCumMerits_t_AssistMngr_Tanner,RCumMerits_t_RetailAssoc_Jake,RCumMerits_t_RetailAssoc_James,RCumMerits_t_RetailAssoc_Jane,RCumMerits_t_RetailAssoc_John,RCumMerits_t_RetailAssoc_Jim,RCumMerits_t_RetailAssoc_Jenny,RCumMerits_t_RetailAssoc_Jeremy,RCumMerits_t_RetailAssoc_Judy,RCumMerits_t_RetailAssoc_Julie,RCumMerits_t_RetailAssoc_Jeffrey,RComplete_t_Manager_Matt,RComplete_t_AssistMngr_Mike,RComplete_t_AssistMngr_Tanner,RComplete_t_RetailAssoc_Jake,RComplete_t_RetailAssoc_James,RComplete_t_RetailAssoc_Jane,RComplete_t_RetailAssoc_John,RComplete_t_RetailAssoc_Jim,RComplete_t_RetailAssoc_Jenny,RComplete_t_RetailAssoc_Jeremy,RComplete_t_RetailAssoc_Judy,RComplete_t_RetailAssoc_Julie,RComplete_t_RetailAssoc_Jeffrey,DSlot_t_Manager,DSlot_t_AssistMngr,DSlot_t_RetailAssoc,DBusy_t_Manager,DBusy_t_AssistMngr,DBusy_t_RetailAssoc,DVolume_t_Manager,DVolume_t_AssistMngr,DVolume_t_RetailAssoc,DRevenue_t_Manager,DRevenue_t_AssistMngr,DRevenue_t_RetailAssoc,DMax_t_Manager,DMax_t_AssistMngr,DMax_t_RetailAssoc,B_t,V_t,N_t,Ucum_Manager,Ucum_AssistMngr,Ucum_RetailAssoc,Ucum_Total,Ccum_CumSlots,Ccum_SickProb,Ccum_CumMerits,Ccum_ContSlots,Ccum,Allocd_t_Manager_Matt___Manager,Allocd_t_AssistMngr_Mike___AssistMngr,Allocd_t_AssistMngr_Tanner___AssistMngr,Allocd_t_RetailAssoc_Jake___RetailAssoc,Allocd_t_RetailAssoc_James___RetailAssoc,Allocd_t_RetailAssoc_Jane___RetailAssoc,Allocd_t_RetailAssoc_John___RetailAssoc,Allocd_t_RetailAssoc_Jim___RetailAssoc,Allocd_t_RetailAssoc_Jenny___RetailAssoc,Allocd_t_RetailAssoc_Jeremy___RetailAssoc,Allocd_t_RetailAssoc_Judy___RetailAssoc,Allocd_t_RetailAssoc_Julie___RetailAssoc,Allocd_t_RetailAssoc_Jeffrey___RetailAssoc
0,X__Alloc,"(0, 0, 0, 1, random)",1,0,2023-12-04 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0.0000,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,False,False,False,False,False,False,False,False,False,False,False,False,False
1,X__Alloc,"(0, 0, 0, 1, random)",1,1,2023-12-04 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0.0000,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,False,False,False,False,False,False,False,False,False,False,False,False,False
2,X__Alloc,"(0, 0, 0, 1, random)",1,2,2023-12-04 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-2,1,0,0,1,-2,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0.0000,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,False,False,False,False,False,False,False,False,False,False,False,False,False
3,X__Alloc,"(0, 0, 0, 1, random)",1,3,2023-12-04 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,-2,1,0,0,2,-2,0,0,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0.0000,0,0,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,False,False,False,False,False,False,False,False,False,False,False,False,False
4

In [ ]:
df = Df_evalu_opt

,dt,Allocd_t_Manager_Matt___Manager,Allocd_t_AssistMngr_Mike___AssistMngr,Allocd_t_AssistMngr_Tanner___AssistMngr,Allocd_t_RetailAssoc_Jake___RetailAssoc,Allocd_t_RetailAssoc_James___RetailAssoc,Allocd_t_RetailAssoc_Jane___RetailAssoc,Allocd_t_RetailAssoc_John___RetailAssoc,Allocd_t_RetailAssoc_Jim___RetailAssoc,Allocd_t_RetailAssoc_Jenny___RetailAssoc,Allocd_t_RetailAssoc_Jeremy___RetailAssoc,Allocd_t_RetailAssoc_Judy___RetailAssoc,Allocd_t_RetailAssoc_Julie___RetailAssoc,Allocd_t_RetailAssoc_Jeffrey___RetailAssoc
0,2023-12-04 00:00:00,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2023-12-04 01:00:00,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2023-12-04 02:00:00,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2023-12-04 03:00:00,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2023-12-04 04:00:00,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
# def prepare_schedule_shifts(df, buf, pars)
# sched = defaultdict(list)
# sched = defaultdict(str)
# sched = {"shifts": {}}
# sched = {"shifts": {}}
# sched = defaultdict(lambda: defaultdict(dict))
# sched_dict = defaultdict(lambda: defaultdict(list))
sched_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

mask = df.columns.str.contains('Allocd_t')
resource_allocs = list(df.columns[mask])
sched = copy(df)
schedule = sched[['dt']+resource_allocs]

print(f"SCHEDULE SHIFTS:")
print(f"================")
for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    # print(f'\n************** {resName}:\n')
    # sched['shifts'].append(resName)
    sched_list = list(schedule.loc[
        schedule[res_alloc] == True,
        ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
        ts_1 = sched_list[0]
        dow_1 = sched_list[0].day_of_week
        # print(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
        # sched['shifts'][resName].append(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        # sched_dict['shifts'][resName].append(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        sched_dict['shifts'][resName][dow_1].append(f"{(sched_list[0]-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
        for ts in sched_list:
            dow = ts.day_of_week
            if dow != dow_1:
                # print(f"{(ts_1).strftime('%a %b %d %Hh%M')}\n\n")
                # sched_dict['shifts'][resName].append(f"{(ts_1).strftime('%a %b %d %Hh%M')}")
                sched_dict['shifts'][resName][dow].append(f"{(ts_1).strftime('%a %b %d %Hh%M')}")
                # sched_dict['shifts'][resName].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
                sched_dict['shifts'][resName][dow].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
                # pass
            dow_1 = dow
            ts_1 = ts
        # print(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}\n")
        # sched_dict['shifts'][resName].append(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}")
        sched_dict['shifts'][resName][dow_1].append(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}")
print(f'\n{sched_dict=}\n')           
print(f'\n{cf.CONTIGUOUS_REWARD=}\n')
print(f'{pars.MAX_DAILY_SLOT_RUN=} ({pars.RESOLUTION}s)\n')
print(f'{cf.TH_CumSlots_SPEC=}\n')
print(f'{cf.TH_SickProb_SPEC=}\n')
print(f'{cf.TH_CumMerits_SPEC=}\n')
print(f'{cf.TH_ContSlots_SPEC=}\n')
print(f'{cf.TH_Select_SPEC=}\n')


SCHEDULE SHIFTS:

sched_dict=defaultdict(<function <lambda>>, {'shifts': defaultdict(<function <lambda>.<locals>.<lambda>>, {'Manager_Matt': defaultdict(<class 'list'>, {0: ['Mon Dec 04 09h00'], 1: ['Mon Dec 04 17h00', 'Tue Dec 05 09h00'], 2: ['Tue Dec 05 17h00', 'Wed Dec 06 09h00'], 3: ['Wed Dec 06 17h00', 'Thu Dec 07 09h00'], 4: ['Thu Dec 07 17h00', 'Fri Dec 08 09h00'], 5: ['Fri Dec 08 17h00', 'Sat Dec 09 09h00'], 6: ['Sat Dec 09 17h00', 'Sun Dec 10 09h00', 'Sun Dec 10 17h00']}), 'AssistMngr_Mike': defaultdict(<class 'list'>, {0: ['Mon Dec 04 17h00'], 1: ['Mon Dec 04 21h00', 'Tue Dec 05 17h00'], 2: ['Tue Dec 05 21h00', 'Wed Dec 06 17h00'], 3: ['Wed Dec 06 21h00', 'Thu Dec 07 09h00'], 4: ['Thu Dec 07 17h00', 'Fri Dec 08 09h00'], 5: ['Fri Dec 08 17h00', 'Sat Dec 09 13h00'], 6: ['Sat Dec 09 20h00', 'Sun Dec 10 14h00', 'Sun Dec 10 19h00']}), 'AssistMngr_Tanner': defaultdict(<class 'list'>, {0: ['Mon Dec 04 09h00'], 1: ['Mon Dec 04 17h00', 'Tue Dec 05 09h00'], 2: ['Tue Dec 05 17h00', 'Wed

In [ ]:
# def prepare_schedule_slots(df, buf)
# sched = defaultdict(list)
# sched = defaultdict(str)
# sched = {"shifts": {}}
# sched = {"shifts": {}}
# sched = defaultdict(lambda: defaultdict(dict))
# sched_dict = defaultdict(lambda: defaultdict(list))
sched_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

gap_mins = utl.gap_minutes(pars.RESOLUTION)
mask = df.columns.str.contains('Allocd_t')
resource_allocs = list(df.columns[mask])
sched = copy(df)
schedule = sched[['dt']+resource_allocs]

# print(f"SCHEDULE SLOTS:")
# print(f"================")
n_gaps = 0
for res_alloc in resource_allocs:
    _,_,id,resType,_,_,_ = res_alloc.split('_')
    resName = id+'_'+resType
    # print(f'\n************** {resName}:\n')
    sched_list = list(schedule.loc[
        schedule[res_alloc] == True,
        ['dt', res_alloc]
    ]['dt'])
    if len(sched_list) > 0:
        ## print each slot and indicate gaps
        ts_1 = sched_list[0]
        dow_1 = -1
        for ts in sched_list:
            dow = ts.day_of_week
            # if dow != dow_1: buf.write('\n')
            if (int((ts.to_datetime64() - ts_1.to_datetime64())/(1e9*60)) > gap_mins)\
                and (dow == dow_1):
                n_gaps += 1
                # buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP\n")
                # sched_dict['shifts'][resName].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP")
                sched_dict['slots'][resName][dow].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')} GAP")
            else:
                # buf.write(f"{(ts-pd.Timedelta(cf.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}\n")
                # sched_dict['shifts'][resName].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
                sched_dict['slots'][resName][dow].append(f"{(ts-pd.Timedelta(pars.DATE_TIME_DELTA)).strftime('%a %b %d %Hh%M')}")
            dow_1 = dow
            ts_1 = ts
        # print(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}\n")
        # sched_dict['shifts'][resName].append(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}")
        # sched_dict['slots'][resName][dow].append(f"{(sched_list[-1]).strftime('%a %b %d %Hh%M')}")       
# print(f'\n{cf.CONTIGUOUS_REWARD=}\n')
sched_dict['config']['CONTIGUOUS_REWARD'] = cf.CONTIGUOUS_REWARD
# print(f'{pars.MAX_DAILY_SLOT_RUN=} ({pars.RESOLUTION}s)\n')
sched_dict['config']['MAX_DAILY_SLOT_RUN'] = pars.MAX_DAILY_SLOT_RUN
# print(f'{cf.TH_CumSlots_SPEC=}\n')
sched_dict['config']['TH_CumSlots_SPEC'] = cf.TH_CumSlots_SPEC
# print(f'{cf.TH_SickProb_SPEC=}\n')
sched_dict['config']['TH_SickProb_SPEC'] = cf.TH_SickProb_SPEC
# print(f'{cf.TH_CumMerits_SPEC=}\n')
sched_dict['config']['TH_CumMerits_SPEC'] = cf.TH_CumMerits_SPEC
# print(f'{cf.TH_ContSlots_SPEC=}\n')
sched_dict['config']['TH_ContSlots_SPEC'] = cf.TH_ContSlots_SPEC
# print(f'{cf.TH_Select_SPEC=}\n')
sched_dict['config']['TH_Select_SPEC'] = cf.TH_Select_SPEC
print(f'\n{sched_dict=}\n')


sched_dict=defaultdict(<function <lambda>>, {'slots': defaultdict(<function <lambda>.<locals>.<lambda>>, {'Manager_Matt': defaultdict(<class 'list'>, {0: ['Mon Dec 04 09h00', 'Mon Dec 04 10h00', 'Mon Dec 04 11h00', 'Mon Dec 04 12h00', 'Mon Dec 04 13h00', 'Mon Dec 04 14h00', 'Mon Dec 04 15h00', 'Mon Dec 04 16h00'], 1: ['Tue Dec 05 09h00', 'Tue Dec 05 10h00', 'Tue Dec 05 11h00', 'Tue Dec 05 12h00', 'Tue Dec 05 13h00', 'Tue Dec 05 14h00', 'Tue Dec 05 15h00', 'Tue Dec 05 16h00'], 2: ['Wed Dec 06 09h00', 'Wed Dec 06 10h00', 'Wed Dec 06 11h00', 'Wed Dec 06 12h00', 'Wed Dec 06 13h00', 'Wed Dec 06 14h00', 'Wed Dec 06 15h00', 'Wed Dec 06 16h00'], 3: ['Thu Dec 07 09h00', 'Thu Dec 07 10h00', 'Thu Dec 07 11h00', 'Thu Dec 07 12h00', 'Thu Dec 07 13h00', 'Thu Dec 07 14h00', 'Thu Dec 07 15h00', 'Thu Dec 07 16h00'], 4: ['Fri Dec 08 09h00', 'Fri Dec 08 10h00', 'Fri Dec 08 11h00', 'Fri Dec 08 12h00', 'Fri Dec 08 13h00', 'Fri Dec 08 14h00', 'Fri Dec 08 15h00', 'Fri Dec 08 16h00'], 5: ['Sat Dec 09 09h00',

In [ ]:
# \\\\\\\\\\

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()